### create and use our own MCP Server and MCP Client!

using accounts.py here

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

In [ ]:
from accounts import Account

In [ ]:
account = Account.get("vinamra")
account

In [ ]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

In [ ]:
account.report()

In [ ]:
account.list_transactions()

### Now we write an MCP server and use it directly!

In [ ]:
# use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [ ]:
mcp_tools

In [ ]:
import os
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel

# OpenRouter configuration
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", os.getenv("OPENAI_API_KEY"))

# Create OpenRouter client
openrouter_client = AsyncOpenAI(
    base_url=OPENROUTER_BASE_URL,
    api_key=OPENROUTER_API_KEY
)

# Create the model instance
mimo_model = OpenAIChatCompletionsModel(
    model="xiaomi/mimo-v2-flash:free",
    openai_client=openrouter_client
)


In [ ]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is vinamra and my account is under the name vinamra. What's my balance and my holdings?"
model = mimo_model


In [ ]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


### Build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
request = "My name is vinamra and my account is under the name vinamra. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
with trace("buy_shares"):
    request = "Buy 20 shares of MSFT for my account under the name vinamra."
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("vinamra")
print(context)

In [ ]:
from accounts import Account
Account.get("vinamra").report()

## Datetime server and client

In [ ]:
from datetime_client import get_datetime_tools_openai

datetime_tools =  await get_datetime_tools_openai()
print(datetime_tools)

In [ ]:
with trace("buy_shares"):
    request = "Tell me current date and time IST"
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=[*openai_tools, *datetime_tools])
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))